# Kaggle Titanic : Toujours plus loin !

In [2]:
import pandas as pd
import re
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier

train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")
full = pd.concat([train, test]) # Assemble les deux jeux de données

# Récupère le la localisation de la cabine babord ou tribord)
https://fr.wikipedia.org/wiki/Passerelle_de_navigation_du_Titanic

L'évacuation des passagers dans les canots est organisée ainsi : le premier officier William Murdoch est chargé de tous les canots situés à tribord (c'est-à-dire tous les canots portant un chiffre impair en plus des canots A et C) et le deuxième officier Charles Lightoller est chargé de tous les canots situés à bâbord (tous les canots portant un chiffre pair en plus des canots B et D)33. Les autres officiers se doivent d'assister Murdoch et Lightoller dans leurs tâches. À 0 h 55, le troisième officier Pitman aide au remplissage du canot no 5 puis embarque dans celui-ci pour en prendre le commandement34.

In [3]:
X = train
X['CabinNumber'] = 0
X['CabinBaTri'] = 0
X['Cabin'] = X['Cabin'].fillna('X0')
for idx, cab in enumerate(X['Cabin']):
    cb = cab.split(" ")[0]
    if cb[1:].isnumeric():
        numcab = int(cb[1:])
    else:
        numcab = 0
    # Récupère le numéro de cabine
    X.loc[idx, 'CabinNumber']= numcab
    
    # Affecte pair (0) ou Impair (1) / babord/Tribord
    if (numcab > 0): 
        X.loc[idx, 'CabinBaTri'] = int(numcab % 2)
    else:
        X.loc[idx, 'CabinBaTri'] = 2

X.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,CabinNumber,CabinBaTri
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,X0,S,0,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,85,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,X0,S,0,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,123,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,X0,S,0,2


# Création d'une fonction de préparation

In [4]:
# Création d'un DataFrame avec tous les noms de famille (full)
familynames = []
for noms in full["Name"]:
    familynames.append(re.search('([A-Za-z0-9]*),\ ([A-Za-z0-9 ]*)\. (.*)', noms).group(1))
pdfamilynames = pd.DataFrame(familynames, columns = ['familynames'])

# Créé une liste filtrée des noms de famille avec plus de 2 occurences !
famsurv = full.join(pdfamilynames)
famCount = famsurv['familynames'].value_counts()
pdfamCounts = pd.DataFrame(famCount, columns = ['familynames'])
pdfamCounts['famCount'] = pdfamCounts['familynames']
pdfamCounts['familynames'] = pdfamCounts.index
pdfamCounts[pdfamCounts['famCount'] >= 2]

# Fonction ajoutant les colonnes noms famille dans un DF
def addColumnFamilyName(data):
    # ajoute les colonnes nulles avec les noms de famille
    for family in pdfamCounts['familynames']:
        data[family] = 0
    # récupère le nom de famille dans le DF
    for idx, f in enumerate(data["Name"]):
        # Modifie les colonnes dummies du nom de famille en 1 ou 0 selon le nom de famille
        iNom = re.search('([A-Za-z0-9]*),\ ([A-Za-z0-9 ]*)\. (.*)', f).group(1)
        for col in data.columns:
            if (col == iNom):
                data.loc[idx, col] = 1

# Fonction globale de préparation : Retourne le DataFrame préparé
def featureEngineering(data):
    # Sexe
    sexe = pd.get_dummies(data['Sex'], prefix='sex')
    
    # Cabine, récupération du pont (on remplace le pont T proche du pont A)
    cabin = pd.get_dummies(data['Cabin'].fillna('X').str[0].replace('T', 'A'), prefix='Cabin')
    
    data['CabinNumber'] = 0
    data['CabinBaTri'] = 0
    data['Cabin'] = data['Cabin'].fillna('X0')
    for idx, cab in enumerate(data['Cabin']):
        cb = cab.split(" ")[0]
        if cb[1:].isnumeric():
            numcab = int(cb[1:])
        else:
            numcab = 0
        # Récupère le numéro de cabine
        data.loc[idx, 'CabinNumber']= numcab

        # Affecte pair (0) ou Impair (1) / babord/Tribord
        if (numcab > 0): 
            data.loc[idx, 'CabinBaTri'] = int(numcab % 2)
        else:
            data.loc[idx, 'CabinBaTri'] = 2
    
    # Age et catégories d'age
    age = data['Age'].fillna(data['Age'].median()) # Age médian de 3eme classe
    catAge = []
    for i in range(data.shape[0]) :
        if age[i] < 3:
            catAge.append("bebe")
        elif age[i] >= 3 and age[i] < 15:
            catAge.append("enfant")
        elif age[i] >= 15 and age[i] < 60:
            catAge.append("adulte") 
        else:
            catAge.append("vieux")
    catage = pd.get_dummies(pd.DataFrame(catAge, columns = ['catAge']), prefix='catAge')
    
    # Titre et Rang
    data['Titre'] = data.Name.map(lambda x : x.split(",")[1].split(".")[0]).fillna('X')
    data['Rang'] = 0
    vip = ['Don','Sir', 'Major', 'Col', 'Jonkheer', 'Dr']
    femmeenfant = ['Miss', 'Mrs', 'Lady', 'Mlle', 'the Countess', 'Ms', 'Mme', 'Dona', 'Master']
    for idx, titre in enumerate(data['Titre']): 
        if (titre.strip() in femmeenfant) :
            data.loc[idx, 'Rang'] = 'FE'
        elif (titre.strip() in vip) :
            data.loc[idx, 'Rang'] = 'VIP'
        else :
            data.loc[idx, 'Rang'] = 'Autres'
    rg = pd.get_dummies(data['Rang'], prefix='Rang')
    
    # Embarquement
    emb = pd.get_dummies(data['Embarked'], prefix='emb')
    
    # Prix unitaire - Ticket, Prépartion d'un DF (TicketCounts) contenant les ticket avec leur nb d'occurence
    TicketCounts = pd.DataFrame(data['Ticket'].value_counts().head())
    TicketCounts['TicketCount'] = TicketCounts['Ticket'] # renomme la colonne Ticket
    TicketCounts['Ticket'] = TicketCounts.index # rajoute une colonne Ticket pour le merge (jointure)
    # reporte le résultat dans le dataframe test (jointure des datasets)
    fin = pd.merge(data, TicketCounts, how='left', on='Ticket')
    fin['PrixUnitaire'] = fin['Fare'] / fin['TicketCount'].fillna(1)
    prxunit = pd.DataFrame(fin['PrixUnitaire'])
    # Prix moyen 3eme classe (pour le passager de 3eme qui n'a pas de prix) ... on aurait pu faire une fonction ici ;-)
    prx3eme = data.loc[data['Pclass'] == 3]['Fare'].mean()
    prxunit = prxunit['PrixUnitaire'].fillna(prx3eme)
    
    # Classe
    pc = pd.DataFrame(MinMaxScaler().fit_transform(data[['Pclass']]), columns = ['Classe'])
    
    # Constitution du DataFrame
    dp = data[['SibSp', 'Parch', 'Name', 'CabinBaTri']].join(pc).join(sexe).join(emb).join(prxunit).join(cabin).join(age).join(catage).join(rg)
    addColumnFamilyName(dp)
    del dp['Name']
    
    return dp

In [5]:
Xtrain = featureEngineering(train)
Xtest = featureEngineering(test)

## Entrainement du modèle

In [60]:
y = train.Survived

#rf = RandomForestClassifier(n_estimators=1000, random_state=3, max_features=1)
#rf = RandomForestClassifier(n_estimators=1500, random_state=5, max_features=20, criterion='gini')
#rf.fit(Xtrain, y)
#p_tr = rf.predict(Xtrain)
#print ("Score Train -->", round(rf.score(Xtrain, y) *100,4), " %")

gb = GradientBoostingClassifier(n_estimators=1000, learning_rate=0.5, max_depth=20, random_state=3)
gb.fit(Xtrain, y)
p_gb = rf.predict(Xtrain)
print ("Score Train -->", round(gb.score(Xtrain, y) *100,4), " %")

Score Train --> 99.8878  %


## Formattage des résultats pour Kaggle ;-)

In [61]:
p_test = rf.predict(Xtest)
result = pd.DataFrame(test['PassengerId'])
pred = pd.DataFrame(p_test, columns=['Survived'])
result = result.join(pred)

In [62]:
result.describe()

,PassengerId,Survived
count,418.000000,418.000000
mean,1100.500000,0.373206
std,120.810458,0.484236
min,892.000000,0.000000
25%,996.250000,0.000000
50%,1100.500000,0.000000
75%,1204.750000,1.000000
max,1309.000000,1.000000


In [63]:
result.to_csv("./data/result78.csv", columns=["PassengerId", "Survived"], index=False)

# Score : 78%